In [ ]:
'''
Okay the plan;



1. We predict with sahi on the whole ortho image using different parameters (overlap, slice sizing) 
3. export each prediction as a seperate folder / yolo file format
2. We tile each prediction result image and bounding boxes into smaller images
    - this would basically be a dataset of original sized tile images along with a corresponding labels folder and a YAML file
3. randomly select 10% of images and label them in labelstudio
4. use fiftyone merge datasets to merge predict output tiles and labels with labelstudio groundtruth label and tiles
5. evaluate model performace and adjust confidence intervals
6. georef and make figures using the whole image predictions




'''

In [5]:
# first check the wd
# this should be AIggregation folder

import os
os.getcwd()

#os.chdir("..")


'/home/tmueller/github/AIggregation'

In [4]:

# import fiftyone
import fiftyone as fo
from fiftyone import ViewField as F


# Import ortho image into fiftyone dataset type
dataset = fo.Dataset.from_images(
    ["datasets/drone_ortho/ortho_clip_23april.png"]
)


# open test image instance to verify it loaded correctly
session = fo.launch_app(dataset)

 100% |█████████████████████| 1/1 [22.2ms elapsed, 0s remaining, 50.7 samples/s] 


In [ ]:
# import sahi 
from sahi import AutoDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction


# specify AI detection model
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov5',
    model_path='AIggregation_yolov5m/weights/best.pt', #specify path to trained model
    confidence_threshold=0.25,
    device="cpu", # or 'cuda:0'
)


In [ ]:


# Run and compare different slicing parameters
def predict_with_slicing(sample, label_field, **kwargs):
    result = get_sliced_prediction(
        sample.filepath, detection_model, verbose=0, **kwargs
    )
    sample[label_field] = fo.Detections(detections=result.to_fiftyone_detections())


#kwargs if you want to specify many arguments that are the same for all models
#kwargs = {"overlap_height_ratio": 0.2, "overlap_width_ratio": 0.2}

kwargs = {}


for sample in dataset.iter_samples(progress=True, autosave=True):
    predict_with_slicing(sample,
                         label_field="small_slices",
                         slice_height=304,
                         slice_width=304,overlap_height_ratio = .2,
                         overlap_width_ratio=.2,
                         **kwargs
    )
    predict_with_slicing(sample,
                         label_field="large_slices",
                         slice_height=608, slice_width=608,
                         overlap_height_ratio = .2, overlap_width_ratio=.2,
                         **kwargs
    )
    predict_with_slicing(sample,
                         label_field="large_slices_large_overlap",
                         slice_height=608, slice_width=608,
                         overlap_height_ratio = .4, overlap_width_ratio=.4,
                         **kwargs
    )


#bring up new session to see boxes
session.show()



In [ ]:
# export the different sahi predictions and save them

In [36]:
# tile the image and outputs


''' 
the exported sahi prediction can now be tiled. 

The below script takes a folder containing identially named image and txt files and tiles them into a new folder. 
This was written for yolov4 format which is slightly different but works for this

'''


# run yolo_tile script
%run scripts/yolo_tile.py -source ./datasets/export_test/tile_test -target ./datasets/tiled_export/tile_test -ext .JPG -size 608 -ratio 0.1

'''
    -source Source folder with images and labels needed to be tiled. Default: ./yolosample/ts/
    -target Target folder for a new sliced dataset. Default: ./yolosliced/ts/
    -ext Image extension in a dataset. Default: .JPG
    -falsefolder Folder for tiles without bounding boxes
    -size Size of a tile. Default: 416
    -ratio Train/test split ratio. Dafault: 0.8
'''

classes.names not found. It should be located one level higher than images
Image: ./datasets/export_test/test.JPG
./datasets/tiled_export/test_0_0.JPG
   class        x1        y1         w         h
0      0  0.984545  0.050868  0.027620  0.101736
1      0  0.312624  0.596640  0.095373  0.116574
2      0  0.487485  0.684600  0.118692  0.118692
3      0  0.985605  0.818124  0.025500  0.135648
./datasets/tiled_export/test_0_1.JPG
   class        x1        y1         w         h
0      0  0.045771  0.050868  0.091543  0.101736
1      0  0.048950  0.818124  0.097901  0.135648
./datasets/tiled_export/test_0_3.JPG
   class        x1        y1         w         h
0      0  0.242826  0.242682  0.114453  0.112332
./datasets/tiled_export/test_0_5.JPG
   class        x1        y1        w         h
0      0  0.697207  0.474768  0.12717  0.139884
1      0  0.344731  0.821358  0.13320  0.115218
./datasets/tiled_export/test_0_6.JPG
   class        x1        y1         w         h
0      0  0.297021

'\n    -source Source folder with images and labels needed to be tiled. Default: ./yolosample/ts/\n    -target Target folder for a new sliced dataset. Default: ./yolosliced/ts/\n    -ext Image extension in a dataset. Default: .JPG\n    -falsefolder Folder for tiles without bounding boxes\n    -size Size of a tile. Default: 416\n    -ratio Train/test split ratio. Dafault: 0.8\n'

In [51]:
'''
reimport as individual fiftyone files

then merge files with different labels



'''

#first split them into different folders and create a yaml file 
# CODE TO COME SOON




#then import in fiftyone

import fiftyone as fo
import fiftyone.utils.yolo as fouy


# Import annotated test image into fiftyone dataset type
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv5Dataset,
    yaml_path = "datasets/tiled_export/tile_test/dataset.yaml",
    label_field= "predict"
)


dataset2 = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv5Dataset,
    yaml_path = "datasets/tiled_export/tile_test/dataset.yaml",
    label_field= "test"
)


# View summary info about the dataset



# below exports labels only for a single label class
dataset.export(
    dataset_type=fo.types.YOLOv5Dataset,
    labels_path="datasets/tiled_export/tmp",
    label_field="predict"
)


fouy.add_yolo_labels(
    dataset2,
    "predict",
    "datasets/tiled_export/tmp",
    classes,
)





session = fo.launch_app(dataset2)

 100% |███████████████████| 38/38 [81.7ms elapsed, 0s remaining, 465.0 samples/s]  
 100% |███████████████████| 38/38 [60.4ms elapsed, 0s remaining, 628.8 samples/s]     
Directory 'datasets/tiled_export/tmp' already exists; export will be merged with existing files
 100% |███████████████████| 38/38 [27.0ms elapsed, 0s remaining, 1.4K samples/s] 


NameError: name 'classes' is not defined

In [ ]:
# okay new test of using fiftyone for splits






In [ ]:

# evaluate 
'''


# evaluate the different sahi predictions
large_slice_results = dataset.evaluate_detections("large_slices", gt_field="ground_truth", eval_key="eval_large_slices")
small_slice_results = dataset.evaluate_detections("small_slices", gt_field="ground_truth", eval_key="eval_small_slices")
overlap_slice_results = dataset.evaluate_detections("large_slices_overlap", gt_field="ground_truth", eval_key="eval_large_slices_overlap")

# print eval results
print("-" * 50)
print("Large slice results:")
large_slice_results.print_report()

print("-" * 50)
print("Large slice overlap results:")
overlap_slice_results.print_report()

print("-" * 50)
print("Small slice results:")
small_slice_results.print_report()

'''